In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant
import enchant
!pip install wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet 
from textblob import Word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,056 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,483 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Pa

[nltk_data] Downloading package wordnet to /root/nltk_data...


Trainin %6 sı test ile birleştirilerek toplam datanın %20 si test olarak ayarlandı

In [3]:
df2 = pd.read_fwf("TrainDataset.txt",header=None)

In [4]:
df2_label= pd.read_fwf("Labels.txt",header=None)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test ,y_train, y_test = train_test_split(df2, df2_label, test_size=0.06, random_state=105)

In [6]:
df3 = pd.read_fwf("TestDataset.txt",header=None)

In [7]:
df3 = df3.append(X_test)

<ipython-input-7-b5f9bae92d7a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(X_test)


In [8]:
df3.rename(columns = {0:'Tweet'}, inplace = True)

In [32]:
df3.to_csv('TestDataset.csv',index=None, mode='a')

In [10]:
X_train.rename(columns = {0:'Tweet'}, inplace = True)

In [11]:
X_train['Label'] = y_train

In [29]:
X_train.to_csv('TrainDataset.csv',index=None, mode='a')

In [30]:
y_train.to_csv('TrainLabel.csv',index=None, mode='a')

Read data

In [33]:
df_test = pd.read_csv("TestDataset.csv")

In [34]:
df_train = pd.read_csv("TrainDataset.csv")

In [35]:
df_train.shape

(346748, 2)

In [36]:
df_test.shape

(72133, 1)

In [37]:
df_train

,Tweet,Label
0,missing chicago @ Downtown Chicago,10
1,Oh did I mention I want a cow or two? Specific...,1
2,#mylove @ Central Park in NYC,1
3,I'm just extremely thankful you're a part of m...,3
4,"First pit stop mommy crib @ Monterey West, Mia...",19
...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,0
346744,Dem boys!!! @ The Beach...point Plesant Nj.,1
346745,@user it's fun to shop tho...I might have to r...,5
346746,Oh gosh look meh people!!!!!! Missed these guy...,9


In [38]:
df_test

,Tweet
0,en Pelham Parkway
1,The calm before...... | w/ sofarsounds @user |...
2,Just witnessed the great solar eclipse @ Tampa...
3,This little lady is 26 weeks pregnant today! E...
4,"Great road trip views! @ Shartlesville, Pennsy..."
...,...
72128,Thank u alvaradopinup #starwars #emilyaddison ...
72129,Seniors! St. Thomas On-Site event is tomorrow!...
72130,Throwback to the best time ever. Take me back ...
72131,For unto us a child was born and what a joy to...


In [39]:
df = pd.DataFrame(df_train, columns=['Tweet'])

In [40]:
#df_train.rename(columns = {0:'Tweets'}, inplace = True)

In [41]:
df.shape

(346748, 1)

In [42]:
df

,Tweet
0,missing chicago @ Downtown Chicago
1,Oh did I mention I want a cow or two? Specific...
2,#mylove @ Central Park in NYC
3,I'm just extremely thankful you're a part of m...
4,"First pit stop mommy crib @ Monterey West, Mia..."
...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...
346744,Dem boys!!! @ The Beach...point Plesant Nj.
346745,@user it's fun to shop tho...I might have to r...
346746,Oh gosh look meh people!!!!!! Missed these guy...


In [43]:
df.Tweet[1]

'Oh did I mention I want a cow or two? Specifically that white one in the middle. Swoon!!! …'

In [44]:
# basic data cleaning 
def cleanTweet (twt):
  twt = re.sub("#\w+\s*", "", twt)
  #twt = re.sub("#","", twt) # remove the '#' from the tweets
  twt = re.sub("@[A-Za-z0-9_]+","", twt) # remove @mentions
  twt = re.sub('[^\w\s]','',twt)  #noktalama
  twt = re.sub('\d+','',twt) #rakam
  twt = re.sub('\n',' ',twt).replace('\r',' ')
  twt = re.sub('RT', '', twt) # remove 'RT' from tweets
  twt = re.sub('\n', '', twt) # remove the '\n' character
  twt = re.sub(r"(?:@|http?://|https?://|www)\S+", "", twt) # remove the hyperlinks
  twt = re.sub('^[\s]+|[\s]+$', '', twt) # remove leading and trailing whitespaces
  twt = twt.lower()
  return twt

In [45]:
# tweets have been tokenized to use in the further steps.
def tokenization(twt):
  words = word_tokenize(twt)
  list = []
  for w in words:
    list.append(w)
  twt = list
  return twt

In [46]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [47]:
# removing stop words 
def removeStopWord (twt):
  stop_words = nltk.corpus.stopwords.words('english')
  filtered_list = []
  for w in twt:
    if w.lower() not in stop_words:
        filtered_list.append(w)
  filtered_list
  twt = filtered_list
  return twt

In [48]:
# 'in' suffix in words fixed with 'ing'
def checkIngSuffix(twt):
  d = enchant.Dict("en_US")
  list = []
  for w in twt:
    if not d.check(w) and w.endswith("in"):
      tmp = w + 'g'
      if d.check(tmp):
        w = tmp
    list.append(w)
  twt = list
  return twt

In [49]:
# repeating letters removed
def removeRepeatingLetter(twt):
  list = []
  for w in twt:
    reps = 2   # number of consecutive repeated chars allowed
    w = ''.join(['' if i>reps-1 and e==w[i-reps] else e for i,e in enumerate(w)])
    list.append(w)
  twt = list
  return twt

In [50]:
# stop words that not in the nltk stop word list have been removed.
def removeStopWordV2 (twt):

  delete_list = ["im", "i'll", "i've", "id", "hes","he's","hed","he'd" "he'll","shes","she's","shed", "she'd","she'll","tis","youre","yall","youl","you'll","youd","you'd","youve","you've",
                 "whos","wheres","whats", "whers","wheres","is","are"]
  
  filtered_list = []
  for w in twt:
    if w not in delete_list:
        filtered_list.append(w)
  filtered_list
  twt = filtered_list
  return twt

In [51]:
# abbreviations converted into their original forms using dictionary.
def changeWords(twt):

  

  my_dict = {'ny' : 'new york',
              'anotha' : 'another',
              'fav' : 'favourite',
              'yep' : 'yes',
              'bf' : 'best friend',
              'sis' : 'sister',
              'tho' : 'though',
              'trina' : 'trying',
              'tb' : 'tbt'
              }
  twt = " ".join(twt)
  for key, value in my_dict.items():
    if twt == key: 
      twt = twt.replace(key, value)
  return twt

In [52]:
df["FirstCleaning"] = df.Tweet.apply(cleanTweet)

In [53]:
df

,Tweet,FirstCleaning
0,missing chicago @ Downtown Chicago,missing chicago downtown chicago
1,Oh did I mention I want a cow or two? Specific...,oh did i mention i want a cow or two specifica...
2,#mylove @ Central Park in NYC,central park in nyc
3,I'm just extremely thankful you're a part of m...,im just extremely thankful youre a part of my ...
4,"First pit stop mommy crib @ Monterey West, Mia...",first pit stop mommy crib monterey west miami...
...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,loved it thank you atlanta progpower usa
346744,Dem boys!!! @ The Beach...point Plesant Nj.,dem boys the beachpoint plesant nj
346745,@user it's fun to shop tho...I might have to r...,its fun to shop thoi might have to remind you ...
346746,Oh gosh look meh people!!!!!! Missed these guy...,oh gosh look meh people missed these guys sooo...


In [54]:
df["TokenizedTweets"] = df.FirstCleaning.apply(tokenization)

In [55]:
df

,Tweet,FirstCleaning,TokenizedTweets
0,missing chicago @ Downtown Chicago,missing chicago downtown chicago,"[missing, chicago, downtown, chicago]"
1,Oh did I mention I want a cow or two? Specific...,oh did i mention i want a cow or two specifica...,"[oh, did, i, mention, i, want, a, cow, or, two..."
2,#mylove @ Central Park in NYC,central park in nyc,"[central, park, in, nyc]"
3,I'm just extremely thankful you're a part of m...,im just extremely thankful youre a part of my ...,"[im, just, extremely, thankful, youre, a, part..."
4,"First pit stop mommy crib @ Monterey West, Mia...",first pit stop mommy crib monterey west miami...,"[first, pit, stop, mommy, crib, monterey, west..."
...,...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,loved it thank you atlanta progpower usa,"[loved, it, thank, you, atlanta, progpower, usa]"
346744,Dem boys!!! @ The Beach...point Plesant Nj.,dem boys the beachpoint plesant nj,"[dem, boys, the, beachpoint, plesant, nj]"
346745,@user it's fun to shop tho...I might have to r...,its fun to shop thoi might have to remind you ...,"[its, fun, to, shop, thoi, might, have, to, re..."
346746,Oh gosh look meh people!!!!!! Missed these guy...,oh gosh look meh people missed these guys sooo...,"[oh, gosh, look, meh, people, missed, these, g..."


In [56]:
# df["NonStopWordTweets"] = df.TokenizedTweets.apply(removeStopWord)

In [58]:
df["FixedIngSuffixTweets"] = df.TokenizedTweets.apply(checkIngSuffix)

In [59]:
df

,Tweet,FirstCleaning,TokenizedTweets,FixedIngSuffixTweets
0,missing chicago @ Downtown Chicago,missing chicago downtown chicago,"[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]"
1,Oh did I mention I want a cow or two? Specific...,oh did i mention i want a cow or two specifica...,"[oh, did, i, mention, i, want, a, cow, or, two...","[oh, did, i, mention, i, want, a, cow, or, two..."
2,#mylove @ Central Park in NYC,central park in nyc,"[central, park, in, nyc]","[central, park, in, nyc]"
3,I'm just extremely thankful you're a part of m...,im just extremely thankful youre a part of my ...,"[im, just, extremely, thankful, youre, a, part...","[im, just, extremely, thankful, youre, a, part..."
4,"First pit stop mommy crib @ Monterey West, Mia...",first pit stop mommy crib monterey west miami...,"[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, mommy, crib, monterey, west..."
...,...,...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,loved it thank you atlanta progpower usa,"[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]"
346744,Dem boys!!! @ The Beach...point Plesant Nj.,dem boys the beachpoint plesant nj,"[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]"
346745,@user it's fun to shop tho...I might have to r...,its fun to shop thoi might have to remind you ...,"[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re..."
346746,Oh gosh look meh people!!!!!! Missed these guy...,oh gosh look meh people missed these guys sooo...,"[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, these, g..."


In [60]:
df["RepeatingRemovedTweets"] = df.FixedIngSuffixTweets.apply(removeRepeatingLetter)

In [61]:
df

,Tweet,FirstCleaning,TokenizedTweets,FixedIngSuffixTweets,RepeatingRemovedTweets
0,missing chicago @ Downtown Chicago,missing chicago downtown chicago,"[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]"
1,Oh did I mention I want a cow or two? Specific...,oh did i mention i want a cow or two specifica...,"[oh, did, i, mention, i, want, a, cow, or, two...","[oh, did, i, mention, i, want, a, cow, or, two...","[oh, di, i, mention, i, want, a, cow, or, two,..."
2,#mylove @ Central Park in NYC,central park in nyc,"[central, park, in, nyc]","[central, park, in, nyc]","[central, park, in, nyc]"
3,I'm just extremely thankful you're a part of m...,im just extremely thankful youre a part of my ...,"[im, just, extremely, thankful, youre, a, part...","[im, just, extremely, thankful, youre, a, part...","[im, just, extremly, thankful, youre, a, part,..."
4,"First pit stop mommy crib @ Monterey West, Mia...",first pit stop mommy crib monterey west miami...,"[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, momy, crib, montery, west, ..."
...,...,...,...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,loved it thank you atlanta progpower usa,"[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]"
346744,Dem boys!!! @ The Beach...point Plesant Nj.,dem boys the beachpoint plesant nj,"[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]"
346745,@user it's fun to shop tho...I might have to r...,its fun to shop thoi might have to remind you ...,"[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re..."
346746,Oh gosh look meh people!!!!!! Missed these guy...,oh gosh look meh people missed these guys sooo...,"[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, thes, gu..."


In [62]:
df["RearrangedTweets"] = df.RepeatingRemovedTweets.apply(removeStopWordV2)

In [63]:
df

,Tweet,FirstCleaning,TokenizedTweets,FixedIngSuffixTweets,RepeatingRemovedTweets,RearrangedTweets
0,missing chicago @ Downtown Chicago,missing chicago downtown chicago,"[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]"
1,Oh did I mention I want a cow or two? Specific...,oh did i mention i want a cow or two specifica...,"[oh, did, i, mention, i, want, a, cow, or, two...","[oh, did, i, mention, i, want, a, cow, or, two...","[oh, di, i, mention, i, want, a, cow, or, two,...","[oh, di, i, mention, i, want, a, cow, or, two,..."
2,#mylove @ Central Park in NYC,central park in nyc,"[central, park, in, nyc]","[central, park, in, nyc]","[central, park, in, nyc]","[central, park, in, nyc]"
3,I'm just extremely thankful you're a part of m...,im just extremely thankful youre a part of my ...,"[im, just, extremely, thankful, youre, a, part...","[im, just, extremely, thankful, youre, a, part...","[im, just, extremly, thankful, youre, a, part,...","[just, extremly, thankful, a, part, of, my, li..."
4,"First pit stop mommy crib @ Monterey West, Mia...",first pit stop mommy crib monterey west miami...,"[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, momy, crib, montery, west, ...","[first, pit, stop, momy, crib, montery, west, ..."
...,...,...,...,...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,loved it thank you atlanta progpower usa,"[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]"
346744,Dem boys!!! @ The Beach...point Plesant Nj.,dem boys the beachpoint plesant nj,"[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]"
346745,@user it's fun to shop tho...I might have to r...,its fun to shop thoi might have to remind you ...,"[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re..."
346746,Oh gosh look meh people!!!!!! Missed these guy...,oh gosh look meh people missed these guys sooo...,"[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, thes, gu...","[oh, gosh, look, meh, people, missed, thes, gu..."


In [64]:
df["RearrangedTweets"] = df.RearrangedTweets.apply(changeWords)

In [65]:
df

,Tweet,FirstCleaning,TokenizedTweets,FixedIngSuffixTweets,RepeatingRemovedTweets,RearrangedTweets
0,missing chicago @ Downtown Chicago,missing chicago downtown chicago,"[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]","[missing, chicago, downtown, chicago]",missing chicago downtown chicago
1,Oh did I mention I want a cow or two? Specific...,oh did i mention i want a cow or two specifica...,"[oh, did, i, mention, i, want, a, cow, or, two...","[oh, did, i, mention, i, want, a, cow, or, two...","[oh, di, i, mention, i, want, a, cow, or, two,...",oh di i mention i want a cow or two specifcall...
2,#mylove @ Central Park in NYC,central park in nyc,"[central, park, in, nyc]","[central, park, in, nyc]","[central, park, in, nyc]",central park in nyc
3,I'm just extremely thankful you're a part of m...,im just extremely thankful youre a part of my ...,"[im, just, extremely, thankful, youre, a, part...","[im, just, extremely, thankful, youre, a, part...","[im, just, extremly, thankful, youre, a, part,...",just extremly thankful a part of my life plain...
4,"First pit stop mommy crib @ Monterey West, Mia...",first pit stop mommy crib monterey west miami...,"[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, mommy, crib, monterey, west...","[first, pit, stop, momy, crib, montery, west, ...",first pit stop momy crib montery west miami fl...
...,...,...,...,...,...,...
346743,Loved it!!! Thank you Atlanta!!! ️ #progpoweru...,loved it thank you atlanta progpower usa,"[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]","[loved, it, thank, you, atlanta, progpower, usa]",loved it thank you atlanta progpower usa
346744,Dem boys!!! @ The Beach...point Plesant Nj.,dem boys the beachpoint plesant nj,"[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]","[dem, boys, the, beachpoint, plesant, nj]",dem boys the beachpoint plesant nj
346745,@user it's fun to shop tho...I might have to r...,its fun to shop thoi might have to remind you ...,"[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re...","[its, fun, to, shop, thoi, might, have, to, re...",its fun to shop thoi might have to remind you ...
346746,Oh gosh look meh people!!!!!! Missed these guy...,oh gosh look meh people missed these guys sooo...,"[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, these, g...","[oh, gosh, look, meh, people, missed, thes, gu...",oh gosh look meh people missed thes guys soo m...


In [81]:
df2 = df["RearrangedTweets"]
df2.to_csv(r'NormalizedTrainDataset2.csv', header=None, index=None, mode='a')

In [87]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:

!python -m spacy download en_core_web_sm

2023-06-01 14:20:29.282818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [103]:
!pip install git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


In [104]:
!git clone https://github.com/shredder47/Nonspaced-Sentence-Tokenizer.git

Cloning into 'Nonspaced-Sentence-Tokenizer'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 195 (delta 28), reused 174 (delta 19), pack-reused 0
Receiving objects: 100% (195/195), 1.59 MiB | 5.22 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [105]:
!pip install nltk
!pip install regex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
df3 = pd.read_csv("NormalizedTrainDataset.csv",header=None)

In [68]:
df3

,0
0,missing chicago downtown chicago
1,oh di i mention i want a cow or two specifcall...
2,central park in nyc
3,just extremly thankful a part of my life plain...
4,first pit stop momy crib montery west miami fl...
...,...
346743,loved it thank you atlanta progpower usa
346744,dem boys the beachpoint plesant nj
346745,its fun to shop thoi might have to remind you ...
346746,oh gosh look meh people missed thes guys soo m...


In [69]:
df_test

,Tweet
0,en Pelham Parkway
1,The calm before...... | w/ sofarsounds @user |...
2,Just witnessed the great solar eclipse @ Tampa...
3,This little lady is 26 weeks pregnant today! E...
4,"Great road trip views! @ Shartlesville, Pennsy..."
...,...
72128,Thank u alvaradopinup #starwars #emilyaddison ...
72129,Seniors! St. Thomas On-Site event is tomorrow!...
72130,Throwback to the best time ever. Take me back ...
72131,For unto us a child was born and what a joy to...


In [70]:
df_test["FirstCleaning"] = df_test.Tweet.apply(cleanTweet)

In [71]:
df_test["TokenizedTweets"] = df_test.FirstCleaning.apply(tokenization)

In [72]:
df_test["FixedIngSuffixTweets"] = df_test.TokenizedTweets.apply(checkIngSuffix)

In [73]:
df_test["RepeatingRemovedTweets"] = df_test.FixedIngSuffixTweets.apply(removeRepeatingLetter)

In [74]:
df_test["RearrangedTweets"] = df_test.RepeatingRemovedTweets.apply(removeStopWordV2)

In [75]:
df_test["RearrangedTweets"] = df_test.RearrangedTweets.apply(changeWords)

In [80]:
df2 = df_test["RearrangedTweets"]
df2.to_csv(r'NormalizedTestDataset2.csv', header=None, index=None, mode='a')

In [79]:
df3 = pd.read_csv("NormalizedTestDataset.csv",header=None)

FileNotFoundError: ignored

In [78]:
df3

,0
0,en pelham parkway
1,the calm before w sofarsounds b hall
2,just witnessed the great solar eclipse tampa f...
3,this little lady weeks pregnat today excited f...
4,great road trip views shartlesville pennsylvania
...,...
72128,thank u alvardopinup aliso
72129,seniors st thomas onsite evnt tomrrow all docs...
72130,throwback to the best time evr take me back sa...
72131,for unto us a child was born and what a joy to...
